# Automated ML

In the cell below, we import all the dependencies that we will need to complete the project.

In [1]:
import joblib
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.widgets import RunDetails

## Dataset

### Overview

We used heart failure dataset(https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). Downloaded this dataset from the kaggle website and uploaded to Azure ML Studio and registered it. This data generally contains data about people and has health related data with the death column. Generally, we have to predict early detection of Heart rate failure using ML.

Columns in dataset are - age,	anaemia	creatinine_phosphokinase,	diabetes,	ejection_fraction,	high_blood_pressure	platelets,	serum_creatinine,	serum_sodium,	sex	smoking	time,	DEATH_EVENT.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.
Reason for ML - Need early detection and management wherein a machine learning model can be of great help.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'
experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,min_node_count=None,timeout_in_minutes=10,
    )


print(f'compute target: {compute_target.get_status().serialize()}')

Found existing compute target: AmlCompute(workspace=Workspace.create(name='quick-starts-ws-262203', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-262203'), name=compute-cluster, id=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourceGroups/aml-quickstarts-262203/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-262203/computes/compute-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags={})
compute target: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-07-11T04:18:34.755000+00:00', 'errors': None, 'creationTime': '2024-07-11T04:18:26.716450+00:00', 'modifiedTime': '2024-07-11T04:18:36.735884+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTim

In [3]:
dataset_name = 'heart-beat'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [4]:
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Field to predict - DEATH_EVENT which is a classification problem, we choose accuracy to be our primary metric here, as data is already registered we have the training dataset ready to use AutoML on it. 

"primary_metric": "accuracy" = This specifies that the primary metric used to evaluate the performance of the models is "accuracy". Accuracy is the ratio of the number of correct predictions to the total number of predictions. It is commonly used for classification tasks where the model needs to predict discrete labels.

"experiment_timeout_minutes": 15 = This sets a time limit for the entire AutoML experiment, which is 15 minutes in this case. The experiment will run for a maximum of 15 minutes. If it doesn't finish within this time, it will stop regardless of whether it has tested all possible configurations or not. This is useful for ensuring experiments do not run indefinitely and helps in managing computational resources.

"max_concurrent_iterations": 5 = This specifies the maximum number of iterations (model training runs) that can be executed concurrently. Running multiple iterations in parallel can speed up the experiment by utilizing available computational resources efficiently. In this case, up to 5 iterations can run simultaneously.

In [6]:
target_column = "DEATH_EVENT"

# automl experiment settings here
automl_settings = {
    "primary_metric": "accuracy",
    "experiment_timeout_minutes": 18,
    "max_concurrent_iterations": 5,
    
}

# automl config here
automl_config = AutoMLConfig(
    compute_target= compute_target,
    training_data= dataset,
    task= "classification",
    label_column_name= target_column,
    path= "./automl-run",
    featurization= "auto",
    debug_log= "automl_errors.log",
    enable_early_stopping= True,
    **automl_settings
    )

In [7]:
# submit the experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_143ba73d-8d16-4983-b4ce-0ca57dfcd14e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

2024-07-11 04:32:35.704261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38/lib/python3.9/site-packages/cv2/../../lib64:
2024-07-11 04:32:35.704310: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_143ba73d-8d16-4983-b4ce-0ca57dfcd14e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+=============

{'runId': 'AutoML_143ba73d-8d16-4983-b4ce-0ca57dfcd14e',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-07-11T04:22:21.508677Z',
 'endTimeUtc': '2024-07-11T04:43:14.010972Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-262203","workspace_name":"quick-starts-ws-262203","region":"westeurope","compute_target":"compute-

## Best Model

Get the best model from the automl experiments and display all the properties of the model.

In [11]:
best_run, best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print(f"Best Model Run ID: {best_run.id}\n")
print(f"Best Model Metrics: {best_run_metrics}\n")
print(f"Best Model Accuracy: {best_run_metrics['accuracy']}\n")

Best Model Run ID: AutoML_143ba73d-8d16-4983-b4ce-0ca57dfcd14e_48

Best Model Metrics: {'accuracy': 0.8931034482758621, 'log_loss': 0.353441052644468, 'average_precision_score_micro': 0.9292984353526347, 'matthews_correlation': 0.7650568173624568, 'AUC_macro': 0.9235436718443012, 'recall_score_micro': 0.8931034482758621, 'AUC_weighted': 0.9235436718443012, 'precision_score_weighted': 0.9051633886547679, 'f1_score_micro': 0.8931034482758621, 'precision_score_macro': 0.8997582096266308, 'balanced_accuracy': 0.8678931332778586, 'recall_score_macro': 0.8678931332778586, 'norm_macro_recall': 0.7357862665557173, 'f1_score_weighted': 0.8902325025572008, 'precision_score_micro': 0.8931034482758621, 'AUC_micro': 0.9273452239397543, 'average_precision_score_weighted': 0.9357174038086257, 'f1_score_macro': 0.8729045755516814, 'recall_score_weighted': 0.8931034482758621, 'weighted_accuracy': 0.9089548460902614, 'average_precision_score_macro': 0.9167355168705992, 'accuracy_table': 'aml://artifactI

In [12]:
print(f"More details on best Model {best_model}")

More details on best Model Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook262203/code/Users/odl_user_262203')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('29', Pipeli...ample_bytree=1, eta=0.4, gamma=0.1, max_depth=10, max_leaves=127, n_estimators=50, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0.8333333333333334, reg_lambda=1.1458333333333335, subsample=0.8, tree_method='auto'))]))], flatten_transform=False, weights=[0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]))])


In [13]:
# save the best model
best_run = automl_run.get_best_child()
best_model_name = best_run.properties["model_name"]

In [14]:
best_run.download_files("./outputs")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

We will deploy the autoML model as accuracy is greater than hyperparamter tune.

In [15]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/99fa57a0/classes.interpret.json',
 'explanation/99fa57a0/eval_data_viz.interpret.json',
 'explanation/99fa57a0/expected_values.interpret.json',
 'explanation/99fa57a0/features.interpret.json',
 'explanation/99fa57a0/global_names/0.interpret.json',
 'explanation/99fa57a0/global_rank/0.interpret.json',
 'explanation/99fa57a0/global_values/0.interpret.json',
 'explanation/99fa57a0/local_importance_values.interpret.json',
 'explanation/99fa57a0/per_class_names/0.interpret.json',
 'explanation/99fa57a0/per_class_rank/0.interpret.json',
 'explanation/99fa57a0/per_class_values/0.interpret.json',
 'explanation/99fa57a0/rich_metadata.interpret.json',
 'explanation/99fa57a0/true_ys_viz.interpret.json',
 'explanation/99fa57a0/visualization_dict.interpret.json',
 'explanation/99fa57a0/ys_pred_proba_viz.interpret.json',
 'explanation/99fa57a0/ys_pred_viz.interpret.json',
 'explanation/debcdd1c/classes.interpret.json',
 'expl

In [16]:
# registering the model
from azureml.core.resource_configuration import ResourceConfiguration
best_model_name = best_run.properties["model_name"]
model = best_run.register_model(
    model_name=best_model_name, 
    model_path="./outputs",
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1),
)

print(f"Model name: {model.name}")
print(f"Version: {model.version}")
print(f"RunID: {model.run_id}")

Model name: AutoML143ba73d848
Version: 1
RunID: AutoML_143ba73d-8d16-4983-b4ce-0ca57dfcd14e_48


In [21]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

inference_env = Environment.get(ws, "AzureML-AutoML")


inference_config = InferenceConfig(entry_script="./score.py", environment=inference_env)


service = Model.deploy(ws, "heart-beat-endpoint", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
service.scoring_uri

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-11 05:53:39+00:00 Creating Container Registry if not exists.
2024-07-11 05:53:40+00:00 Registering the environment.
2024-07-11 05:53:40+00:00 Use the existing image.
2024-07-11 05:53:40+00:00 Generating deployment configuration.
2024-07-11 05:53:43+00:00 Submitting deployment to compute.
2024-07-11 05:53:49+00:00 Checking the status of deployment heart-beat-endpoint..
2024-07-11 05:58:36+00:00 Checking the status of inference endpoint heart-beat-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


'http://6dd7cb43-96ce-4ace-93ed-5fb8e2ef9555.westeurope.azurecontainer.io/score'

In [22]:
service.update(enable_app_insights=True)
service.get_logs()

'2024-07-11T05:58:22,130365100+00:00 - rsyslog/run \n2024-07-11T05:58:22,142447600+00:00 - gunicorn/run \n2024-07-11T05:58:22,148293700+00:00 | gunicorn/run | \n2024-07-11T05:58:22,156224700+00:00 | gunicorn/run | ###############################################\n2024-07-11T05:58:22,159238300+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-07-11T05:58:22,160715400+00:00 - nginx/run \n2024-07-11T05:58:22,162171100+00:00 | gunicorn/run | ###############################################\n2024-07-11T05:58:22,165759000+00:00 | gunicorn/run | \n2024-07-11T05:58:22,183753700+00:00 | gunicorn/run | \n2024-07-11T05:58:22,208711100+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240624.v1\n2024-07-11T05:58:22,221312600+00:00 | gunicorn/run | \n2024-07-11T05:58:22,238508700+00:00 | gunicorn/run | \n2024-07-11T05:58:22,248830500+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [28]:
import urllib.request
import json
import os

data =  {
  "data": [
    {
      "age": 25.0,
      "anaemia": 0,
      "creatinine_phosphokinase": 0,
      "diabetes": 1,
      "ejection_fraction": 0,
      "high_blood_pressure": 1,
      "platelets": 0.0,
      "serum_creatinine": 0.0,
      "serum_sodium": 0,
      "sex": 1,
      "smoking": 1,
      "time": 0
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'http://6dd7cb43-96ce-4ace-93ed-5fb8e2ef9555.westeurope.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[1]'


In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
